# Tasca M6 T01
## Exercici 1
Crea almenys dos models de regressió diferents per intentar predir el millor possible el preu de les vivendes (MEDV) de l'arxiu adjunt.

Reutilizaré el procesamiento de datos realizado en el [ejercicio anterior](https://github.com/epili50/M5-T02.git)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = df = pd.read_csv('housing data.csv', sep = ',', header = None)
df.columns=['CRIM','ZN','INDUS', 'CHAS','NOX','RM', 'AGE','DIS','RAD', 'TAX','PTRATIO','B', 'LSTAT', 'MEDV']

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Proceso los datos para adecuarlos al modelo

In [3]:
from sklearn import preprocessing 

#RobustScaler para las columnas con outliers
columnsR = ['CRIM', 'ZN', 'RM', 'DIS', 'PTRATIO', 'B', 'LSTAT']
scaler = preprocessing.RobustScaler().fit(df[columnsR])
df[columnsR] = scaler.transform(df[columnsR])

#Normalización para las columnas no gaussianas
columnsN = ['INDUS', 'NOX', 'AGE', 'TAX']
scalerNorm = preprocessing.MinMaxScaler().fit(df[columnsN])
df[columnsN] = scalerNorm.transform(df[columnsN])

Divido entre features y target

In [4]:
X = df.drop(['MEDV'],axis=1)
y = df[['MEDV']]

y.head()

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


Divido Train/Test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.80)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(404, 13) (102, 13) (404, 1) (102, 1)


**Modelos**

1º crearé el modelo de: Linear regression [Fuente](https://github.com/aina1997/MasterClassDS/blob/main/Introducci%C3%B3%20a%20la%20Intel%C2%B7lig%C3%A8ncia%20artificial/AI-Regression.ipynb)

In [6]:
#Importo el modelo 
from sklearn.linear_model import LinearRegression

In [7]:
#Instancializo el modelo y lo entreno
lr = LinearRegression().fit(X_train, y_train)

In [8]:
#Defino la variable para predecir el objetivo del test (y_test) 
#dándole la información con la que no se ha entrenado (x_test)
y_pred_lr = lr.predict(X_test)

#Predigo los 10 primeros valores de MEDV (y_test)
y_pred_lr[:10]

array([[24.88963777],
       [23.72141085],
       [29.36499868],
       [12.12238621],
       [21.44382254],
       [19.2834443 ],
       [20.49647539],
       [21.36099298],
       [18.8967118 ],
       [19.9280658 ]])

2º crearé un modelo de Randon Forest [Fuente](https://towardsdatascience.com/random-forest-in-python-24d0893d51c0)

In [9]:
#Importo el modelo 
from sklearn.ensemble import RandomForestRegressor

In [10]:
#Instancializo el modelo y lo entreno
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

RandomForestRegressor()

In [11]:
#Defino la variable para predecir el objetivo del test (y_test) 
#dándole la información con la que no se ha entrenado (x_test)
y_pred_rf = rf.predict(X_test)

#Predigo los 10 primeros valores de MEDV (y_test)
y_pred_rf[:10]

array([24.011, 27.188, 21.845, 10.818, 20.797, 20.577, 21.146, 20.193,
       20.584, 18.819])

## Exercici 2
Compara’ls en base al MSE i al R2.

In [12]:
#Importo el evaluador r2
from sklearn.metrics import r2_score

#Instancializo r2 para ambos modelos
r2_lr = r2_score(y_test, y_pred_lr)
r2_rf = r2_score(y_test, y_pred_rf)

In [13]:
#Importo el evaluador MSE
from sklearn.metrics import mean_squared_error

#Instancializo MSE para ambos modelos
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_rf = mean_squared_error(y_test, y_pred_rf)

In [14]:
metrics = pd.DataFrame( data = [['Linear Regression', r2_lr, mse_lr],
                                  ['Randon Forest', r2_rf, mse_rf]],
                                  columns = ['Model', 'R2', 'MSE'])
metrics.head()

,Model,R2,MSE
0,Linear Regression,0.589222,33.448980
1,Randon Forest,0.746556,20.637534


En esta primera prueba el método de Randon Forest se muestra más eficiente que el de Linear Regression, con un R2 del 72% contra el 58%, y con un error cuadrático más bajo (22 contra 33).

## Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten per intentar millorar-ne la predicció.

Extraje información de la siguiente [Fuente](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74) y del siguiente [Código](https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/random_forest_explained/Improving%20Random%20Forest%20Part%202.ipynb) que pertenece al mismo autor

In [15]:
#parametros de randon forest
rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [16]:
from pprint import pprint

# ver los Parámetros actualmente en uso
print('Parámetros actualmente en uso:\n')
pprint(rf.get_params())

Parámetros actualmente en uso:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [17]:
from sklearn.model_selection import RandomizedSearchCV
import warnings 
warnings.filterwarnings('ignore')

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]


random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

rf = RandomForestRegressor(random_state = 42)

rf_random = RandomizedSearchCV(estimator = rf,
                                param_distributions = random_grid,
                                n_iter = 100, 
                                cv = 5, 
                                scoring='neg_mean_absolute_error', 
                                verbose=2, 
                                random_state=42, 
                                n_jobs = -1)


rf_random.fit(X,y.values.ravel())

rf_random.best_params_


Fitting 5 folds for each of 100 candidates, totalling 500 fits


{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

In [18]:
rf_bp = RandomForestRegressor(n_estimators=800,random_state = 42, min_samples_split=2, min_samples_leaf=1, 
                              max_features='auto', max_depth=100, bootstrap= True).fit(X_train, y_train)
y_pred_rf_bp = rf_bp.predict(X_test)

r2_rf_bp = r2_score(y_test, y_pred_rf_bp)
mse_rf_bp = mean_squared_error(y_test, y_pred_rf_bp)

print(r2_rf_bp, mse_rf_bp)

0.7710926177208358 18.6395708287378


In [19]:
metrics = pd.DataFrame( data = [['Linear Regression', r2_lr, mse_lr],
                                  ['RF', r2_rf, mse_rf],
                                  ['RF Best Parameters', r2_rf_bp, mse_rf_bp]],
                                  columns = ['Model', 'R2', 'MSE'])
metrics.head()

,Model,R2,MSE
0,Linear Regression,0.589222,33.448980
1,RF,0.746556,20.637534
2,RF Best Parameters,0.771093,18.639571


Conclusion: la búsqueda aleatoria de parámetros y su utilización mejora el modelo. 

## Exercici 4
Compara el seu rendiment emprant l’aproximació traint/test o emprant totes les dades (validació interna).

In [20]:
from sklearn.model_selection import cross_val_score

cv_lr = cross_val_score(lr, X, y, cv=5, scoring='r2')
cv_rf = cross_val_score(rf, X, y, cv=5, scoring='r2')
cv_rf_bp = cross_val_score(rf_bp, X, y, cv=5, scoring='r2')

In [21]:
cv_mean = [cv_lr.mean(), cv_rf.mean(), cv_rf_bp.mean()]
metrics['R2 con CV'] = cv_mean
metrics.head()

metrics.head()

,Model,R2,MSE,R2 con CV
0,Linear Regression,0.589222,33.448980,0.353276
1,RF,0.746556,20.637534,0.627563
2,RF Best Parameters,0.771093,18.639571,0.627094


Conclusión: la prueba de Cross Validation muestra que la utilización de mejores parámetros no es eficiente, y que  el modelo de Random Forest en general da buenos resultados. 

Quizás lo recomendable sea buscar otro modelo

## Exercici 5
No facis servir la variable del nombre d'habitacions (RM) a l’hora de fer prediccions.

In [22]:
X2 = df.drop(['RM','MEDV'],axis=1)

X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=0, train_size = 0.80)

print(X2_train.shape, X2_test.shape, y_train.shape, y_test.shape)


(404, 12) (102, 12) (404, 1) (102, 1)


In [23]:
lr2 = LinearRegression().fit(X2_train, y_train)
y_pred_lr2 = lr2.predict(X2_test)
r2_lr2 = r2_score(y_test, y_pred_lr2)
mse_lr2 = mean_squared_error(y_test, y_pred_lr2)

rf2 = RandomForestRegressor().fit(X2_train, y_train)
y_pred_rf2 = rf2.predict(X2_test)
r2_rf2 = r2_score(y_test, y_pred_rf2)
mse_rf2 = mean_squared_error(y_test, y_pred_rf2)


print('El resultado  r2 del modelo Linear Regression sin la variable RM es:', r2_lr2,
      '\nEl resultado  r2 del modelo Random Forest sin la variable RM es:', r2_rf2,
      '\nEl resultado  mse del modelo Linear Regression sin la variable RM es:', mse_lr2,
      '\nEl resultado  mse del modelo Renadom Forest sin la variable RM es:',mse_rf2)

El resultado  r2 del modelo Linear Regression sin la variable RM es: 0.5251737706543536 
El resultado  r2 del modelo Random Forest sin la variable RM es: 0.6181083680511996 
El resultado  mse del modelo Linear Regression sin la variable RM es: 38.66435868126337 
El resultado  mse del modelo Renadom Forest sin la variable RM es: 31.096839480392152


Conclusión: Sin duda la variable RM (Número promedio de habitaciones por vivienda) es importante y quitarla empeora el modelo.